In [1]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestRegressor, BaggingRegressor
from nltk.stem.snowball import SnowballStemmer

In [2]:
stemmer = SnowballStemmer('english')

df_train = pd.read_csv('train.csv', encoding="ISO-8859-1")
df_test = pd.read_csv('test.csv', encoding="ISO-8859-1")
df_pro_desc = pd.read_csv('product_descriptions.csv')

num_train = df_train.shape[0]

def str_stemmer(s):
    return " ".join([stemmer.stem(word) for word in s.lower().split()])

def str_common_word(str1, str2):
    return sum(int(str2.find(word)>=0) for word in str1.split())

In [3]:
df_all = pd.concat((df_train, df_test), axis=0, ignore_index=True)

df_all = pd.merge(df_all, df_pro_desc, how='left', on='product_uid')

df_all['search_term_stemmed'] = df_all['search_term'].map(lambda x:str_stemmer(x))
df_all['product_title_stemmed'] = df_all['product_title'].map(lambda x:str_stemmer(x))
df_all['product_description_stemmed'] = df_all['product_description'].map(lambda x:str_stemmer(x))

In [4]:
df_all['len_of_query'] = df_all['search_term_stemmed'].map(lambda x:len(x.split())).astype(np.int64)

df_all['product_info'] = df_all['search_term_stemmed']+"\t"+df_all['product_title_stemmed']+"\t"+df_all['product_description_stemmed']

df_all['word_in_title'] = df_all['product_info'].map(lambda x:str_common_word(x.split('\t')[0],x.split('\t')[1]))
df_all['word_in_description'] = df_all['product_info'].map(lambda x:str_common_word(x.split('\t')[0],x.split('\t')[2]))

In [17]:
df_features = df_all[[u'id',
     u'relevance',
     u'len_of_query',
     u'word_in_title',
     u'word_in_description']]

df_train = df_features.iloc[:num_train]
df_test = df_features.iloc[num_train:]
id_test = df_test['id']

y_train = df_train['relevance'].values
X_train = df_train.drop(['id','relevance'],axis=1).values
X_test = df_test.drop(['id','relevance'],axis=1).values

In [18]:
rf = RandomForestRegressor(n_estimators=15, max_depth=6, random_state=0)
clf = BaggingRegressor(rf, n_estimators=45, max_samples=0.1, random_state=25)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

pd.DataFrame({"id": id_test, "relevance": y_pred}).to_csv('submission.csv',index=False)